In [1]:
from datetime import datetime
import pandas as pd

In [2]:
import devanalyst.simulation.statics as S_

from devanalyst.simulation.simulationModels import ModelsConfig, DefaultCostModel, MeritocraticCostModel, \
DistributedLagQualityModel, GreedyAllocationModel, NoLaggardsAllocationModel

from devanalyst.simulation.businessObjects import UserStory, Ticket

from devanalyst.metrics.ProductivityMetrics import CommitsAnalysis
import devanalyst.simulation.generateTimecards as timecard

importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\simulationModels.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\businessObjects.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\metrics\ProductivityMetrics.ipynb
importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\generateTimecards.ipynb


In [3]:
import devanalyst.test_utils.test_utils as tu_

importing Jupyter notebook from c:\users\aleja\documents\code\chateauclaudia-labs\devanalyst\devanalyst\test_utils\test_utils.ipynb


In [4]:
DEVA_DF = tu_.DEV_DF[tu_.DEV_DF['Scrum Team'] == 'A']
DEVA_DF

,Name,Developer Profile,Scrum Team,Functional Area,Salary
0,Anton Easterday,Solid,A,Doctor,27000
1,Beau Hockensmith,Mediocre,A,Doctor,20000
2,Bruno Studley,New,A,Doctor,17500
3,Craig Garlitz,New,A,Doctor,17500
4,Francisco Hoppe,Star,A,Patient,35000
5,Glenna Mcghie,Mediocre,A,Patient,20000
6,Gregorio Darr,QA,A,Patient,23000
7,Heriberto Martini,Solid,A,Patient,27000


In [5]:
RELEASE_DURATION = 60
SPRINT_DURATION = 10
NUMBER_OF_SPRINTS = 20

modelsConfig = ModelsConfig([MeritocraticCostModel()], [DistributedLagQualityModel()], NoLaggardsAllocationModel())
modelsConfig.random.reset(271)

teams_df, stories_dfDUMMY, globalRepo = tu_.initTestData(DEVA_DF, tu_.PM_DF, \
                                                    RELEASE_DURATION, SPRINT_DURATION, modelsConfig)
modelsConfig.globalRepo = globalRepo

In [6]:
entries_df, log = timecard.runReleaseCycle(datetime(2018, 1, 15), SPRINT_DURATION, NUMBER_OF_SPRINTS, modelsConfig) 
## Need to set stories3_df **after** release cycle since developer ownerships might be changed during the release
stories_df = UserStory.build_stories_df(globalRepo)
entries_df.shape

(345, 8)

In [7]:
burnout = timecard.releaseBurnout(entries_df)
burnout

,Sprint,Stories Started,Stories Active,Implementing (# stories),Debugging (# stories),Stories Completed,Cum % Completion,Effort,Implementation Effort,Debugging Effort,Active Developers
0,1,27,27,27.0,0.0,5,0.061728,80.000000,80.000000,0.0,8
1,2,12,30,30.0,0.0,6,0.135802,80.000000,80.000000,0.0,8
2,3,5,28,27.0,1.0,5,0.197531,80.000000,79.600000,0.4,8
3,4,4,28,25.0,3.0,5,0.259259,80.000000,77.800000,2.2,8
4,5,6,30,26.0,4.0,7,0.345679,80.000000,73.600000,6.4,8
5,6,5,27,25.0,2.0,2,0.370370,80.000000,76.600000,3.4,8
6,7,6,30,26.0,4.0,8,0.469136,80.000000,69.000000,11.0,8
7,8,5,28,24.0,4.0,5,0.530864,80.000000,73.400000,6.6,8
8,9,3,26,24.0,2.0,7,0.617284,80.000000,73.200000,6.8,8
9,10,5,25,24.0,1.0,6,0.691358,80.000000,79.600000,0.4,8


In [15]:
u = timecard.utilization(entries_df, [11, 12, 13, 14, 15, 16, 17, 18, 19])
u

,Sprint 11,Sprint 12,Sprint 13,Sprint 14,Sprint 15,Sprint 16,Sprint 17,Sprint 18,Sprint 19
Anton Easterday,10.0,10.000000,0.733333,7.333333,NaN,NaN,NaN,NaN,NaN
Beau Hockensmith,10.0,10.000000,10.000000,10.000000,6.8,NaN,NaN,NaN,NaN
Bruno Studley,10.0,10.000000,10.000000,10.000000,2.8,NaN,NaN,NaN,6.0
Craig Garlitz,10.0,10.000000,10.000000,6.800000,NaN,4.8,NaN,NaN,NaN
Francisco Hoppe,10.0,9.633333,9.000000,NaN,NaN,1.6,NaN,NaN,NaN
Glenna Mcghie,10.0,10.000000,10.000000,10.000000,NaN,7.2,NaN,NaN,NaN
Gregorio Darr,10.0,10.000000,7.000000,NaN,NaN,NaN,NaN,NaN,NaN
Heriberto Martini,10.0,10.000000,10.000000,2.600000,NaN,NaN,NaN,NaN,NaN


In [18]:
u.reset_index(inplace = True)
u

,index,Sprint 11,Sprint 12,Sprint 13,Sprint 14,Sprint 15,Sprint 16,Sprint 17,Sprint 18,Sprint 19
0,Anton Easterday,10.0,10.000000,0.733333,7.333333,NaN,NaN,NaN,NaN,NaN
1,Beau Hockensmith,10.0,10.000000,10.000000,10.000000,6.8,NaN,NaN,NaN,NaN
2,Bruno Studley,10.0,10.000000,10.000000,10.000000,2.8,NaN,NaN,NaN,6.0
3,Craig Garlitz,10.0,10.000000,10.000000,6.800000,NaN,4.8,NaN,NaN,NaN
4,Francisco Hoppe,10.0,9.633333,9.000000,NaN,NaN,1.6,NaN,NaN,NaN
5,Glenna Mcghie,10.0,10.000000,10.000000,10.000000,NaN,7.2,NaN,NaN,NaN
6,Gregorio Darr,10.0,10.000000,7.000000,NaN,NaN,NaN,NaN,NaN,NaN
7,Heriberto Martini,10.0,10.000000,10.000000,2.600000,NaN,NaN,NaN,NaN,NaN


In [9]:
workStolen = timecard.tabulateWorkStolen(modelsConfig)
timecard.buildWorkStolen_df(workStolen)

,User Story Id,Assigned on Sprint,Assigned to Developer
0,UserStory #1,9,Anton Easterday
1,UserStory #1,10,Heriberto Martini
2,UserStory #4,7,Beau Hockensmith
3,UserStory #4,8,Heriberto Martini
4,UserStory #5,4,Anton Easterday
5,UserStory #5,5,Heriberto Martini
6,UserStory #7,11,Beau Hockensmith
7,UserStory #7,12,Anton Easterday
8,UserStory #7,13,Francisco Hoppe
9,UserStory #20,6,Anton Easterday


In [10]:
timecard.explainWorkStolen(workStolen, 'UserStory #1', log)

C:\Users\aleja\Documents\CodeImages\Technos\Anaconda3\envs\devanalystenv\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Sprint,Owner,Task Type,Task Description,User Story Id,Planned for Sprint,Most Recently Assigned in Sprint,Delivered in Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved,Global Estimate,% Global Done
2,9,Anton Easterday,UNFINISHED_STORIES,Story implementation,UserStory #1,9.0,9.0,NOT_SET,10.0,NEXT_SPRINT,0.0,10.0,0.000000,10.0,0.00
24,10,Heriberto Martini,UNFINISHED_STORIES,Story implementation,UserStory #1,9.0,10.0,NOT_SET,10.0,CURRENT_SPRINT,5.8,7.1,0.290000,10.0,0.29
18,11,Heriberto Martini,UNFINISHED_STORIES,Story implementation,UserStory #1,9.0,10.0,NOT_SET,7.1,CURRENT_SPRINT,10.0,2.1,0.704225,10.0,0.79
14,12,Heriberto Martini,UNFINISHED_STORIES,Story implementation,UserStory #1,9.0,10.0,12,2.1,CURRENT_SPRINT,4.2,0.0,1.000000,10.0,1.00


In [11]:
timecard.explainWorkStolen(workStolen, 'UserStory #70', log)

,Sprint,Owner,Task Type,Task Description,User Story Id,Planned for Sprint,Most Recently Assigned in Sprint,Delivered in Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved,Global Estimate,% Global Done
18,1,Glenna Mcghie,UNFINISHED_STORIES,Story implementation,UserStory #70,1.0,1.0,NOT_SET,5.000000,CURRENT_SPRINT,0.000000,5.000000,0.000000,5.0,0.000000
21,7,Glenna Mcghie,UNFINISHED_STORIES,Story implementation,UserStory #70,1.0,1.0,NOT_SET,5.000000,CURRENT_SPRINT,0.000000,5.000000,0.000000,5.0,0.000000
7,8,Beau Hockensmith,UNFINISHED_STORIES,Story implementation,UserStory #70,1.0,8.0,NOT_SET,5.000000,CURRENT_SPRINT,0.000000,5.000000,0.000000,5.0,0.000000
7,8,Beau Hockensmith,UNFINISHED_STORIES,Story implementation,UserStory #70,1.0,8.0,NOT_SET,5.000000,CURRENT_SPRINT,0.000000,5.000000,0.000000,5.0,0.000000
15,9,Francisco Hoppe,UNFINISHED_STORIES,Story implementation,UserStory #70,1.0,9.0,NOT_SET,5.000000,CURRENT_SPRINT,1.366667,3.633333,0.273333,5.0,0.273333
12,10,Francisco Hoppe,UNFINISHED_STORIES,Story implementation,UserStory #70,1.0,9.0,10,3.633333,CURRENT_SPRINT,3.633333,0.000000,1.000000,5.0,1.000000
5,13,Francisco Hoppe,DEV_TIME_BUGS,Ticket #27,UserStory #70,13.0,13.0,13,1.000000,CURRENT_SPRINT,1.000000,0.000000,1.000000,1.0,1.000000


In [12]:
lag, bal, act = timecard.workStealingRetrospective(13, log, modelsConfig)
lag

,Owner,Task Type,Task Description,User Story Id,Planned for Sprint,Most Recently Assigned in Sprint,Delivered in Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved,Global Estimate,% Global Done,Sprint
13,Gregorio Darr,UNFINISHED_STORIES,Story implementation,UserStory #73,10,10,NOT_SET,3.666667,CURRENT_SPRINT,0.0,3.666667,0.0,7,0.47619,12


In [13]:
bal

,Owner,Task Type,Task Description,User Story Id,Planned for Sprint,Most Recently Assigned in Sprint,Delivered in Sprint,Original Estimate,Bucket,Effort Spent,Effort Remaining,Percent Achieved,Global Estimate,% Global Done,Sprint
0,Anton Easterday,UNFINISHED_STORIES,Story implementation,UserStory #78,5,11,NOT_SET,5.366667,CURRENT_SPRINT,10.0,0.366667,0.931677,9,0.959259,12
1,Anton Easterday,UNFINISHED_STORIES,Story implementation,UserStory #7,11,12,NOT_SET,8.000000,CURRENT_SPRINT,0.0,8.000000,0.000000,8,0.000000,12
3,Beau Hockensmith,UNFINISHED_STORIES,Story implementation,UserStory #63,8,12,NOT_SET,7.000000,CURRENT_SPRINT,6.0,5.500000,0.214286,9,0.388889,12
4,Beau Hockensmith,UNFINISHED_STORIES,Story implementation,UserStory #40,8,12,NOT_SET,4.500000,NEXT_SPRINT,0.0,4.500000,0.000000,6,0.250000,12
8,Craig Garlitz,UNFINISHED_STORIES,Story implementation,UserStory #81,5,5,NOT_SET,4.666667,CURRENT_SPRINT,2.0,4.000000,0.142857,10,0.600000,12
12,Gregorio Darr,UNFINISHED_STORIES,Story implementation,UserStory #9,10,10,NOT_SET,5.666667,CURRENT_SPRINT,10.0,2.333333,0.588235,9,0.740741,12
13,Gregorio Darr,UNFINISHED_STORIES,Story implementation,UserStory #73,10,10,NOT_SET,3.666667,CURRENT_SPRINT,0.0,3.666667,0.000000,7,0.476190,12
15,Heriberto Martini,UNFINISHED_STORIES,Story implementation,UserStory #51,10,11,NOT_SET,8.000000,CURRENT_SPRINT,5.8,5.100000,0.362500,8,0.362500,12


In [14]:
act

,User Story Id,Assigned on Sprint,Assigned to Developer
8,UserStory #7,13,Francisco Hoppe
21,UserStory #40,13,Glenna Mcghie
51,UserStory #73,13,Glenna Mcghie
